## 1. First set these to the file you want to process and the target of the new prepocessed file

Default mode is to replace the file and save the old file as archival. 
If you provide an outname, the old file is untouched and the new file is saved to the given target.


In [1]:
# Name of the input file
concatenated_nwb = r'C:\Users\guru\Desktop\201931-20190729D1\nwb_201931_20190729_D1\201931_20190729_D1.nwb'
# Name of the output file
# If None, saves over the input concatenated file and renames the raw file as archival
outname = None 

In [2]:
# parameters to be used by the NWB objects
electrode_name = 'e0'
description = 'WPI TM33A20' #'FHC_bipolar electrode'# 
location = 'Vg'
device_name = 'AM Systems 1700 B277 modified with headstage'
num_sites = 1.
x=1.5
y=0.
z=5.9
imp=2.0
filtering = '300-10000 Hz'
#motor_motion_name = 'Port_A_8_bits'
neural_channel_name = 'Neural'

In [3]:
import nwb_wrapper
import pynwb
from pynwb import NWBHDF5IO
import numpy as np
import awake_nwb
import os
data_path = os.path.split(concatenated_nwb)[0]

### Read in the concatenated NWB

In [4]:
io = NWBHDF5IO(concatenated_nwb,'r')
data = io.read()

### Set up a new output data file

In [5]:
data_out = pynwb.NWBFile(data.session_description,
                        data.identifier,
                        data.session_start_time,
                        experimenter=data.experimenter,
                        lab=data.lab,
                        institution=data.institution,
                         experiment_description=data.experiment_description
                        )

### Create electrode and assign it the data from neural

In [6]:
acquisition_list = [x.lower() for x in data.acquisition.keys()]
device = data_out.create_device(name=device_name)

electrode_group = data_out.create_electrode_group(electrode_name,description=description,location=location,device=device)
data_out.add_electrode(0,
                   x=x,
                   y=y,
                   z=z,
                   imp=imp,
                   location=location,
                   filtering=filtering,
                   group=electrode_group)

electrode_table_region = data_out.create_electrode_table_region([0],'The only electrode')
neural = nwb_wrapper.convert_TS_to_ES(data.get_acquisition(neural_channel_name),electrode_table_region)
data_out.add_acquisition(neural)


Front <class 'pynwb.base.TimeSeries'>
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (27504000,), type "<f8">
  description: no description
  interval: 1
  num_samples: 27504000
  resolution: 4.8828125
  timestamps: <HDF5 dataset "timestamps": shape (27504000,), type "<f8">
  timestamps_unit: Seconds
  unit: mV

### Compute the frame times and add to the output file 

In [8]:
if 'sent_trig' in acquisition_list:
    front_samps = awake_nwb.trigger_to_idx(data.get_acquisition('Front').data)
    front_times = data.get_acquisition('Front').timestamps.value[front_samps]

    top_samps = awake_nwb.trigger_to_idx(data.get_acquisition('Top').data)
    top_times = data.get_acquisition('Top').timestamps.value[top_samps]

    frame_samps = awake_nwb.trigger_to_idx(data.get_acquisition('Sent_trig').data)
    frame_times = data.get_acquisition('Sent_trig').timestamps.value[frame_samps]

    # create timeseries
    front_trig = pynwb.TimeSeries('Front',
                           front_samps,
                            timestamps=front_times,
                                  unit-=
                           )
    top_trig = pynwb.TimeSeries('Top',
                           top_samps,
                            timestamps=top_times,
                           )
    cam_trig = pynwb.TimeSeries('Trigger',
                           frame_samps,
                            timestamps=frame_times,
                           )
    
    data_out.add_acquisition(front_trig)
    data_out.add_acquisition(top_trig)
    data_out.add_acquisition(cam_trig)

    #TODO: check if front=top=sent
else:
    frame_samps = awake_nwb.trigger_to_idx(data.get_acquisition('Cam_trig').data)
    frame_times = data.get_acquisition('Cam_trig').timestamps.value[frame_samps]
    cam_trig = pynwb.TimeSeries('Trigger',
                                frame_samps,
                                timestamps=frame_times,
                                unit=data.get_acquisition('Cam_trig').unit)
    data_out.add_acquisition(cam_trig)
    
print('Disregard caluclated frame rate message') # Since we are concatenating across trials, this is calculating during non-recording periods too

Frame rate of camera is calculated to be 947.78 fps

Frame rate of camera is calculated to be 947.78 fps

Frame rate of camera is calculated to be 950.79 fps

Disregard caluclated frame rate message


### Add trials that correspond to the given recordings 

In [9]:
for ii in range(len(data.trials)):
    _,start,stop = data.trials[ii]
    data_out.add_trial(start_time=start,stop_time=stop)

### Save the file. If no output file was given, then save over the input and backup the input as archival

In [10]:
io.close()
if outname is None:
    basename = os.path.splitext(os.path.split(concatenated_nwb)[1])[0]
    os.rename(concatenated_nwb,os.path.join(data_path,basename+'_archive.nwb'))
    outname=concatenated_nwb

In [11]:
with NWBHDF5IO(outname,'w') as io:
    io.write(data_out)

C:\Users\guru\Anaconda2\envs\nwb\lib\site-packages\pynwb\form\build\map.py:882: MissingRequiredWarning: attribute 'unit' for 'data' (TimeSeries)
  warnings.warn(msg, MissingRequiredWarning)
